Importando bibliotecas externas e pacotes implementados

In [1]:
import sys
sys.path.insert(0, '../..')
from util import carregamento_dados as cd, ferramentas_grafos as fg
import pandas as pd
from collections import defaultdict

Importando os 3 arquivos principais:

1. Vínculos
2. Informações gerais das licitações (ano, valor, ...)
3. CNPJs participantes por licitação

In [2]:
dump_path = '../../pickles/licitacoes/'
relacoes_entre_cnpjs = cd.salvar_relacoes_entre_cnpjs()
informacoes_licitacoes = cd.salvar_informacoes_licitacoes()
cnpjs_por_licitacao = cd.salvar_cnpjs_por_licitacao()
%time


CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 8.58 µs


Transformando os dados representados em DataFrames pandas para dicionários nativos (melhor tempo de consulta e construção de dados)

In [3]:
d_relacoes = cd.cnpjs_relacionados_por_cnpj(relacoes_entre_cnpjs)
d_licitacoes = cd.cnpjs_por_licitacao(cnpjs_por_licitacao)
%time

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.58 µs


Criando uma lista com os números identificadores de cada licitação a partir do arquivo de CNPJs participantes por relação.
OBS: Existem licitações no arquivo de informações gerais para as quais não existem informações dos CNPJs participantes.

In [4]:
dados_licitacao = cnpjs_por_licitacao.values
licitacoes = [licitacao for licitacao, _ in dados_licitacao]
%time

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs


Inicializa-se o dicionário aninhado vazio para armazenar os dados das licitações

In [5]:
d = {
    licitacao: {
        'cnpjs' : [],
        'ano' : None,
        'municipio': None,
        'modalidade': None,
        'valor': None,
        'grafo': None,
        'cliques': None
    } for licitacao in licitacoes
}

%time

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.29 µs


In [6]:
for licitacao, cnpj in cnpjs_por_licitacao.values:
    d[licitacao]['cnpjs'].append(cnpj)

%time
    

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


Agora alimentamos o dicionário com os dados de cada licitação: município, modalidade, ano, valor

In [7]:
for l in informacoes_licitacoes.values:
    try:
        d[l[0]]['municipio'] = l[1]
        d[l[0]]['ano'] = l[5]
        d[l[0]]['modalidade'] = l[3]
        d[l[0]]['valor'] = l[7]
    except:
        # Exceção para licitações que não possuem informações de CNPJs licitantes
        pass

%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


Conferindo o dicionário para uma licitação

In [8]:
d['746396']

{'cnpjs': ['19644907000111',
  '19703568000105',
  '19644880000167',
  '21849894000169',
  '21679375000108',
  '19925594000170',
  '21923822000114',
  '21884015000130',
  '19757865000125',
  '19703659000132',
  '19581052000127',
  '07840525000184'],
 'ano': '2015',
 'municipio': 'Cascalho Rico',
 'modalidade': 'Pregão Presencial',
 'valor': '1190070.0',
 'grafo': None,
 'cliques': None}

Para cada licitação cria-se o grafo de CNPJs licitantes e lista-se as cliques

In [9]:
for licitacao in licitacoes:
    grafo = fg.gera_grafo_licitacao(
        licitacao, d_relacoes, d_licitacoes
    )
    cliques = fg.lista_cliques(grafo)

    d[licitacao]['grafo'] = grafo
    d[licitacao]['cliques'] = cliques

%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.58 µs


Como queremos listar cliques individuais com tamanho mínimo de dois CNPJs, alimentamos um dicionário indexado por cada uma dessas cliques.

In [10]:
lu = pd.Series(licitacoes).unique()
lu

array(['761453', '761484', '761487', ..., '933417', '933430', '933667'],
      dtype=object)

In [11]:
# c : cliques dictionary
c = defaultdict(dict)

clique_id = 0
for licitacao in pd.Series(licitacoes).unique():
    for clique in d[licitacao]['cliques']:
        tamanho_clique = len(clique)
        if tamanho_clique >= 2:
            printable_cnpjs = ''
            for cnpj in clique:
                printable_cnpjs += (str(cnpj) + ';')
            c[clique_id] = {
                'ano' : d[licitacao]['ano'],
                'municipio': d[licitacao]['municipio'],
                'tipo_processo_licitatorio': d[licitacao]['modalidade'],
                'id_licitacao' : licitacao,
                'valor': d[licitacao]['valor'],
                'vinculo_em_uso': '1',
                'quantidade_cnpjs': len(d[licitacao]['cnpjs']),
                'quantidade_vinculos': fg.calcula_quantidade_arestas(d[licitacao]['grafo']),
                'densidade_grafo': fg.calcula_densidade(d[licitacao]['grafo']),
                'tam_clique_encontrada': tamanho_clique,
                'lista_de_cnpjs_compondo_clique' : printable_cnpjs,
            }
            clique_id += 1

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


Agora salvamos o dicionário como um dataframe para facilitar a interface com o sistema

In [12]:
cliques = pd.DataFrame.from_dict(
    data=c,
    orient='index'
)

In [13]:
cliques

,ano,municipio,tipo_processo_licitatorio,id_licitacao,valor,vinculo_em_uso,quantidade_cnpjs,quantidade_vinculos,densidade_grafo,tam_clique_encontrada,lista_de_cnpjs_compondo_clique
0,2016,Abaeté,Tomada de Preços,985585,235160.08,1,4,1,0.166667,2,19268374000110;14054958000170;
1,2016,Abaeté,Tomada de Preços,985626,261413.89,1,3,1,0.333333,2,19268374000110;14054958000170;
2,2016,Taiobeiras,Pregão Presencial,933933,NaN,1,3,1,0.333333,2,05459851000110;71400253000109;
3,2017,Abaeté,Pregão Presencial,985641,111523.0,1,3,1,0.333333,2,18210168000197;13668217000116;
4,2017,Abaeté,Tomada de Preços,985654,272170.36,1,4,1,0.166667,2,19268374000110;14054958000170;
...,...,...,...,...,...,...,...,...,...,...,...
5122,2017,Japonvar,Pregão Presencial,811299,372327.1,1,3,1,0.333333,2,17375792000181;00464331000182;
5123,2016,Papagaios,Pregão Presencial,858116,71470.0,1,2,1,1.000000,2,71104624000105;01278828000179;
5124,2015,Matias Barbosa,Pregão Presencial,833610,44569.5,1,2,1,1.000000,2,11178248000163;21308487000144;
5125,2017,Capetinga,Pregão Presencial,736138,73548.0,1,2,1,1.000000,2,26202548000108;17777187000137;


In [15]:
cliques.to_csv('cliques.csv')